In [1]:
# modules we'll use
import pandas as pd
import numpy as np
from numpy.random import rand
import re
import seaborn as sns


# read in all our data
train_data= pd.read_excel("./train.xlsx", engine='openpyxl')
test_data = pd.read_excel("./test.xlsx", engine='openpyxl')



## Preprocess train data

In [2]:

train_data.replace(to_replace=np.nan,value = '*', inplace = True)

In [3]:

#Process margin(d), get the 1st number if there is '/'
DFmargind= pd.DataFrame(train_data['margin(d)'])

i = 0
for i in range(0,DFmargind.shape[0]):
    if '/' in str(DFmargind.iloc[i,0]):
        DFmargind.iloc[i,0] = str(DFmargind.iloc[i,0]).split('/')[0]
    else:
        continue

In [4]:
#Process WBC pod1 to pod 6 for getting a mean
DFWBC = pd.DataFrame(train_data[['WBC_POD#1','WBC_POD#2','WBC_POD#3','WBC_POD#5']])



In [5]:
#Replace '*' with 0
DFWBC.replace(to_replace='*',value = 0, inplace = True)

In [6]:
df_temp = pd.DataFrame(columns = ['WBCzeronumber','WBCmean'],index = range(0,DFWBC.shape[0]))

In [7]:
#Get "0"numbers in each row of WBC and fill it in the temp
i = j = count = 0
for i in range(0,DFWBC.shape[0]):
    for j in range(0,DFWBC.shape[1]):
        if DFWBC.iloc[i,j] == 0:
            count = count + 1
        else:
            continue
    df_temp.iloc[i,0] = count
    count = 0
        

In [8]:
#sum WBC each row,get mean, and fill it in the temp
i = j = sum = mean = 0
for i in range(0,DFWBC.shape[0]):
    for j in range(0,DFWBC.shape[1]):
        sum = sum + DFWBC.iloc[i,j]
        mean = sum/(DFWBC.shape[1]-df_temp.iloc[i,0])
    df_temp.iloc[i,1] = "%.2f"%mean
    sum = 0
    mean = 0

<ipython-input-8-161e67dd7ae7>:6: RuntimeWarning: invalid value encountered in double_scalars
  mean = sum/(DFWBC.shape[1]-df_temp.iloc[i,0])


In [9]:
#Process CRP_POD#1 to CRP_POD#5, to get mean
DFCRP = pd.DataFrame(train_data[['CRP_POD#1','CRP_POD#2','CRP_POD#3','CRP_POD#5']])

#Replace '*' with 0
DFCRP.replace(to_replace='*',value = 0, inplace = True)

df_temp1 = pd.DataFrame(columns = ['CRPzeronumber','CRPmean'],index = range(0,DFWBC.shape[0]))

#Get "0"numbers in each row of CRP and fill it in the temp
i = j = count = 0
for i in range(0,DFCRP.shape[0]):
    for j in range(0,DFCRP.shape[1]):
        if DFCRP.iloc[i,j] == 0:
            count = count + 1
        else:
            continue
    df_temp1.iloc[i,0] = count
    count = 0
        
#sum CRP each row and fill it in the temp
i = j = sum = mean = 0
for i in range(0,DFCRP.shape[0]):
    for j in range(0,DFCRP.shape[1]):
        sum = sum + DFCRP.iloc[i,j]
        mean = sum/(DFCRP.shape[1]-df_temp.iloc[i,0])
    df_temp1.iloc[i,1] = "%.2f"%mean
    sum = 0
    mean = 0

<ipython-input-9-2cc4069cf6a9>:25: RuntimeWarning: invalid value encountered in double_scalars
  mean = sum/(DFCRP.shape[1]-df_temp.iloc[i,0])


In [10]:
#column of WBC_Pre/post, CRP_Pre/post, 위암병소수
DFWBC_Pre = pd.DataFrame(train_data['WBC_Pre'])
DFWBC_Post = pd.DataFrame(train_data['WBC_Post'])
DFCRP_Pre = pd.DataFrame(train_data['CRP_Pre'])
DFCRP_Post = pd.DataFrame(train_data['CRP_Post'])
DF위암병소수 = pd.DataFrame(train_data['위암병소수'])


In [11]:
#numeric columns
numericColumns = (train_data.iloc[:,[3]] #age
                    .join(train_data.iloc[:,[9]]) #BMI
                    .join(train_data.iloc[:,42:44]) #수술시간(min) 출혈량(mL)
                    .join(train_data.iloc[:,51:53]) #GAS OUT SD start
                    .join(train_data.iloc[:,[58]]) #병변1 크기(Cm)
                    .join(DFmargind) #margin(d) after being processed
                    .join(DFWBC_Pre)
                    .join(DFWBC_Post)
                    .join(DFCRP_Pre)
                    .join(DFCRP_Post)
                    .join(df_temp.iloc[:,[1]] ) #WBC mean after being processed
                    .join(df_temp1.iloc[:,[1]] ) #CRP mean after being processed
                    .join(DF위암병소수) #위암병소수

                    )

In [12]:
numericColumns

,나이,BMI,수술시간(min),출혈량(mL),GAS OUT,SD start,병변1 크기(Cm),margin(d),WBC_Pre,WBC_Post,CRP_Pre,CRP_Post,WBCmean,CRPmean,위암병소수
0,53,17.11,180,80,3,4,10.4,1.3,6.39,10.37,0.06,0.03,6.26,5.44,1
1,62,24.01,180,80,2,5,2.3,9.6,6.07,16.72,0.02,0.02,7.99,5.06,1
2,70,29.7,180,50,2,5,1.5,14.8,6.11,12.34,*,0.15,8.43,10.18,1
3,72,25.06,180,100,4,14,1.5,13.1,6.59,6.01,1.34,3.77,7.52,16.45,2
4,53,27.88,180,50,3,4,1.2,11.6,10.14,17.72,*,0.21,9.45,10.54,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,68,24.6,210,100,2,4,3,9.5,5.83,14.6,*,0.52,6.01,22.63,1
541,63,21.49,150,50,4,4,3.5,9.8,8.22,15.24,*,0.12,10.08,8.43,1
542,68,25.88,210,100,2,4,6.5,6.7,4.99,9.45,*,0.67,4.37,18.89,1
543,59,19.97,180,100,4,6,11.2,1.7,8.74,13.25,1.49,1.79,8.07,7.28,1


In [13]:
#categorical columns

categoricalColumns = (train_data.iloc[:,[2]] #gender
                        .join(train_data.iloc[:,[10]]) #smoking
                        .join(train_data.iloc[:,[11]]) #가족암병력
                        .join(train_data.iloc[:,38:42]) #정규/응급	수술명	개복전환	문합법
                        .join(train_data.iloc[:,44:51]) #합병절제	adhesion	invasion	radicality	LN dissection	혈관변이	수혈
                        .join(train_data.iloc[:,54:58]) #병변1 AGC 분류	병변1 EGC 분류	병변1 tubular	병변1 circular
                        .join(train_data.iloc[:,[73]]) #Stage
                        


)

In [14]:
categoricalColumns

,성별,smoking,가족암병력,정규/응급,수술명,개복전환,문합법,합병절제,adhesion,invasion,radicality,LN dissection,혈관변이,수혈,병변1 AGC 분류,병변1 EGC 분류,병변1 tubular,병변1 circular,Stage
0,1,0,0,0,4,0,1,0,1,1,0,4,0,0,3,*,0,2,4
1,1,0,1,0,0,1,1,0,1,1,0,4,0,0,*,3,3,2,1
2,0,0,0,0,9,1,3,0,1,1,0,2,0,0,*,4,2,2,1
3,1,0,0,0,5,0,4,0,1,1,0,4,0,0,*,2,1,0,1
4,1,0,0,0,1,2,4,0,1,1,0,4,0,0,*,4,1,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1,1,0,0,0,1,3,0,1,1,0,4,*,0,*,2,1,0,1
541,0,0,0,0,0,1,3,0,1,1,0,4,*,0,*,4,1,0,2
542,1,0,0,0,0,1,3,0,1,1,0,4,*,0,*,4,1,3,1
543,0,0,0,0,4,0,3,0,1,1,0,4,*,0,3,*,2,0,3


In [15]:
#margin(d) is at 70. USeful columns?
#train1 = train_data.iloc[:,[2,3,9,10,11,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,51,54,55,56,57,58,70,71,73,74,75,84,85]] 


In [16]:
#count * (which means missing value) in each column
numericColumns.isin(['*']).sum()

나이              0
BMI             8
수술시간(min)      20
출혈량(mL)        14
GAS OUT        19
SD start       20
병변1 크기(Cm)     33
margin(d)      30
WBC_Pre         0
WBC_Post        7
CRP_Pre       408
CRP_Post       19
WBCmean         0
CRPmean         0
위암병소수          24
dtype: int64

In [17]:
categoricalColumns.isin(['*']).sum()

성별                           0
smoking                      5
가족암병력                        9
정규/응급                        9
수술명                          7
개복전환                        12
문합법                         29
합병절제                        23
adhesion                    16
invasion                    17
radicality                  30
LN dissection               30
혈관변이                       135
수혈                          12
병변1 AGC 분류                 369
병변1 EGC 분류                 192
병변1 tubular                 33
병변1 circular                44
Stage                       25
dtype: int64

In [18]:
numericColumns.shape[0]

545

In [19]:
#missed 135, 369, and 192 repectively, out of 545, so drop them
categoricalColumns_new = categoricalColumns.drop(['혈관변이','병변1 AGC 분류','병변1 EGC 분류'], axis=1)

In [20]:
numericColumns_new = numericColumns.drop('CRP_Pre', axis=1)

In [21]:
#concat both with DSL and onset
numericColumns_with_result = pd.concat([numericColumns_new,train_data.iloc[:,[-2,-1]]],axis=1)
categoricalColumns_new_with_result = pd.concat([categoricalColumns_new,train_data.iloc[:,[-2,-1]]],axis=1)

In [22]:
#concat numericColumns_with_result and categoricalColumns_new_with_result
train_data_new = pd.concat([numericColumns_with_result,categoricalColumns_new_with_result],axis=1)

In [23]:
#make the last 2 columns a new dataframe
result = train_data_new.iloc[:,-2:]


In [24]:
# convert train_data_new to float type and drop the last 2 columns
train_data_new = train_data_new.drop(train_data_new.iloc[:,-2:],axis=1)

In [25]:
#replace * with NaN

train_data_new.replace(to_replace='*',value = np.nan, inplace = True)



In [26]:
#keep the 1st number before ','
train_data_new = train_data_new.apply(lambda x: x.str.split(',').str[0] if x.dtype == "object" else x)


In [27]:
#drop DSL and onset
train_data_new = train_data_new.drop(train_data_new.iloc[:,-2:],axis=1)

In [28]:
#fill missing values by knn 
from fancyimpute import KNN
train_data_new_filled = pd.DataFrame(KNN(k=10).fit_transform(train_data_new))



Imputing row 1/545 with 4 missing, elapsed time: 0.053
Imputing row 101/545 with 4 missing, elapsed time: 0.058
Imputing row 201/545 with 4 missing, elapsed time: 0.062
Imputing row 301/545 with 4 missing, elapsed time: 0.067
Imputing row 401/545 with 4 missing, elapsed time: 0.071
Imputing row 501/545 with 4 missing, elapsed time: 0.076


In [29]:
#fill missing values by SimpleFill
#from fancyimpute import SimpleFill
#train_data_new_filled = pd.DataFrame(SimpleFill().fit_transform(train_data_new))

In [30]:
# make the last 2 columns of train_data_new_filled a new dataframe
result_filled = train_data_new_filled.iloc[:,-2:]

In [31]:
#drop the last 2 columns of train_data_new_filled
train_data_new_filled = train_data_new_filled.drop(train_data_new_filled.iloc[:,-2:],axis=1)


In [32]:
#replace the column name of train_data_new_filled.columns[0:-2] with train_data_new.columns
train_data_new_filled.columns = train_data_new.columns

ValueError: Length mismatch: Expected axis has 26 elements, new values have 28 elements

In [ ]:
#concat train_data_new_filled and result
train_data_new_filled = pd.concat([train_data_new_filled,result_filled],axis=1)

In [ ]:
#replace * with 0 in DSL and onser 
train_data_new_filled['DSL'].replace(to_replace='*',value = 0, inplace = True)
train_data_new_filled['onset'].replace(to_replace='*',value = 0, inplace = True)


In [ ]:
#export the data to csv
train_data_new_filled.to_csv('train_data_new_filled_5_Dec.csv',index=False)
#export the data to XLSX
train_data_new_filled.to_excel('train_data_new_filled_5_Dec.xlsx',index=False)



## Then let's start processing test data as what we did to train data


In [ ]:
test_data.replace(to_replace=np.nan,value = '*', inplace = True)


In [ ]:

#Process margin(d) in test, get the 1st number if there is '/'
DFmargind_test= pd.DataFrame(test_data['margin(d)'])

i = 0
for i in range(0,DFmargind_test.shape[0]):
    if '/' in str(DFmargind_test.iloc[i,0]):
        DFmargind_test.iloc[i,0] = str(DFmargind_test.iloc[i,0]).split('/')[0]
    else:
        continue

In [ ]:
#Process WBC pod1 to pod 5, get mean
DFWBC_test= pd.DataFrame(test_data[['WBC_POD#1','WBC_POD#2','WBC_POD#3','WBC_POD#5']])

In [ ]:
#Replace '*' with 0
DFWBC_test.replace(to_replace='*',value = 0, inplace = True)

In [ ]:
df_temp2 = pd.DataFrame(columns = ['WBCzeronumber_test','WBCmean_test'],index = range(0,DFWBC_test.shape[0]))


In [ ]:
#Get "0"numbers in each row of WBC_test and fill it in the temp
i = j = count = 0
for i in range(0,DFWBC_test.shape[0]):
    for j in range(0,DFWBC_test.shape[1]):
        if DFWBC_test.iloc[i,j] == 0:
            count = count + 1
        else:
            continue
    df_temp2.iloc[i,0] = count
    count = 0
        

In [ ]:
#sum WBC_test each row and fill it in the temp2
i = j = sum = mean = 0
for i in range(0,DFWBC_test.shape[0]):
    for j in range(0,DFWBC_test.shape[1]):
        sum = sum + DFWBC_test.iloc[i,j]
        mean = sum/(DFWBC_test.shape[1]-df_temp.iloc[i,0])
    df_temp2.iloc[i,1] = "%.2f"%mean
    sum = 0
    mean = 0

In [ ]:
#Process CRP_POD#1 to CRP_POD#5, for getting mean
DFCRP_test = pd.DataFrame(test_data[['CRP_POD#1','CRP_POD#2','CRP_POD#3','CRP_POD#5']])

#Replace '*' with 0
DFCRP_test.replace(to_replace='*',value = 0, inplace = True)

df_temp3 = pd.DataFrame(columns = ['CRPzeronumber_test','CRPmean_test'],index = range(0,DFWBC.shape[0]))

#Get "0"numbers in each row of WBC and fill it in the temp
i = j = count = 0
for i in range(0,DFCRP_test.shape[0]):
    for j in range(0,DFCRP_test.shape[1]):
        if DFCRP_test.iloc[i,j] == 0:
            count = count + 1
        else:
            continue
    df_temp3.iloc[i,0] = count
    count = 0
        
#sum WBC each row and fill it in the temp
i = j = sum = mean = 0
for i in range(0,DFCRP_test.shape[0]):
    for j in range(0,DFCRP_test.shape[1]):
        sum = sum + DFCRP_test.iloc[i,j]
        mean = sum/(DFCRP_test.shape[1]-df_temp3.iloc[i,0])
    df_temp3.iloc[i,1] = "%.2f"%mean
    sum = 0
    mean = 0

In [ ]:

DFWBC_Pre_test = pd.DataFrame(test_data['WBC_Pre'])
DFWBC_Post_test = pd.DataFrame(test_data['WBC_Post'])
DFCRP_Pre_test = pd.DataFrame(test_data['CRP_Pre'])
DFCRP_Post_test = pd.DataFrame(test_data['CRP_Post'])
DF위암병소수_test = pd.DataFrame(test_data['위암병소수'])

In [ ]:
#numerical columns
numericColumns_test = (test_data.iloc[:,[3]] #age
                    .join(test_data.iloc[:,[9]]) #BMI
                    .join(test_data.iloc[:,42:44]) #수술시간(min) 출혈량(mL)
                    .join(test_data.iloc[:,51:53]) #GAS OUT SD start
                    .join(test_data.iloc[:,[58]]) #병변1 크기(Cm)
                    .join(DFmargind_test) #margin(d) after being processed
                    .join(DFWBC_Pre_test)
                    .join(DFWBC_Post_test)
                    .join(DFCRP_Pre_test)
                    .join(DFCRP_Post_test)
                    .join(df_temp2.iloc[:,[1]] ) #WBC mean after being processed
                    .join(df_temp3.iloc[:,[1]] ) #CRP mean after being processed
                    .join(DF위암병소수_test) #위암병소수

                    )

In [ ]:
#categorical columns

categoricalColumns_test = (test_data.iloc[:,[2]] #gender
                        .join(test_data.iloc[:,[10]]) #smoking
                        .join(test_data.iloc[:,[11]]) #가족암병력
                        .join(test_data.iloc[:,38:42]) #정규/응급	수술명	개복전환	문합법
                        .join(test_data.iloc[:,44:51]) #합병절제	adhesion	invasion	radicality	LN dissection	혈관변이	수혈
                        .join(test_data.iloc[:,54:58]) #	병변1 AGC 분류	병변1 EGC 분류	병변1 tubular	병변1 circular
                        .join(test_data.iloc[:,[73]]) #Stage
                        
)

In [ ]:
#drop coloumns what we dropped in train data

In [ ]:
categoricalColumns_test_new = categoricalColumns_test.drop(['혈관변이','병변1 AGC 분류','병변1 EGC 분류'], axis=1)

In [ ]:
numericColumns_test_new = numericColumns_test.drop('CRP_Pre', axis=1)

In [ ]:
#drop margin(d)
#numericColumns_test = numericColumns_test.drop(['margin(d)'],axis = 1)

In [ ]:
#concat numericColumns_test and categoricalColumns_test
test_data = pd.concat([numericColumns_test,categoricalColumns_test],axis = 1)

In [ ]:
#concat both with DSL and onset
numericColumns_test_new_with_result = pd.concat([numericColumns_test_new,test_data["DSL"]]],axis=1)
categoricalColumns_test_new_with_result = pd.concat([categoricalColumns_test_new,test_data["DSL"]]],axis=1)

In [ ]:
test_data_new = pd.concat([categoricalColumns_test_new,numericColumns_test_new,pd.read_excel("./test.xlsx", engine='openpyxl')['DSL']],axis=1)

In [ ]:
#convert * to nan
test_data_new.replace(to_replace='*',value = np.nan, inplace = True)

In [ ]:
test_data_new = test_data_new.apply(lambda x: x.str.split(',').str[0] if x.dtype == "object" else x)


In [ ]:
# convert test_data_new to float type and drop the last column
test_data_new = test_data_new.astype(float)
test_data_new = test_data_new.drop(['DSL'],axis = 1)


In [ ]:
#fill missing values by knn 
from fancyimpute import KNN
test_data_newfilled = pd.DataFrame(KNN(k=10).fit_transform(test_data_new))



In [ ]:
#fill missing values by SimpleFill
#from fancyimpute import SimpleFill
#test_data_new_filled = pd.DataFrame(SimpleFill().fit_transform(test_data_new))

In [ ]:
#recover column names with test_data
test_data_newfilled.columns = test_data_new.columns

In [ ]:
#contact test_data_newfilled with DSL
test_data_newfilled_with_result = pd.concat([test_data_newfilled,pd.read_excel("./test.xlsx", engine='openpyxl')['DSL']],axis=1)

In [ ]:

#export
test_data_newfilled_with_result.to_excel('test_data_filled_5_Dec.xlsx',index = False)

test_data_newfilled_with_result.to_csv('test_data_filled_5_Dec.csv',index = False)